<a href="https://www.kaggle.com/code/andrey36912/notebooke9cda60ba3?scriptVersionId=195207356" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/exam-1/data.csv


Побудова моделі для задач класифікації та регресії

при необхідності згенерувати нові ознаки
самостійно обрати модель для тренування або обрати за допомогою optuna
розділити дані з викидами на тренувальні та тестові, можливо валідаційні якщо потрібно
вибрати метрики для оцінки якості моделі, вибір пояснити
побудувати препроцесор
натренувати модель
вивести метрики для тестових даних, дати оцінку якості моделі
зберегти натреновану модель

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
import optuna
from sklearn.compose import make_column_selector

In [40]:
df = pd.read_csv('/kaggle/input/exam-1/data.csv')

In [41]:
def convert_stay_to_days(stay):
    stay_ranges = {
        '0-10': 5,
        '11-20': 15,
        '21-30': 25,
        '31-40': 35,
        '41-50': 45,
        '51-60': 55,
        '61-70': 65,
        '71-80': 75,
        '81-90': 85,
        '91-100': 95,
        'More than 100 Days': 110
    }
    return stay_ranges.get(stay, np.nan)

df['Stay_Days'] = df['Stay'].apply(convert_stay_to_days)

In [42]:
df = df.drop(columns=['Stay', 'Untitled'], errors='ignore')

In [43]:
y_regression = df['Stay_Days'] 
X = df.drop(columns=['Stay_Days'], errors='ignore')

In [44]:
numerical_features = make_column_selector(dtype_include=np.number)
categorical_features = make_column_selector(dtype_include=object)
numerical_features = numerical_features(X)
categorical_features = categorical_features(X)

In [45]:
for feature in numerical_features:
    X[feature] = X[feature].fillna(X[feature].median())
for feature in categorical_features:
    X[feature] = X[feature].fillna(X[feature].mode()[0])

In [46]:
Q1 = X[numerical_features].quantile(0.25)
Q3 = X[numerical_features].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

X_cleaned = X[~((X[numerical_features] < lower_bound) | (X[numerical_features] > upper_bound)).any(axis=1)]
y_cleaned = y_regression[X_cleaned.index]  # Приведение y к размеру очищенных данных
print("Розмір даних до видалення викидів:", X.shape)
print("Розмір даних після видалення викидів:", X_cleaned.shape)

Розмір даних до видалення викидів: (318438, 20)
Розмір даних після видалення викидів: (270882, 20)


In [47]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [48]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),  
        ('cat', categorical_transformer, categorical_features)  
    ]
)

In [49]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

In [50]:
def objective_regression(trial):
    regressor_name = trial.suggest_categorical('regressor', ['LinearRegression', 'Ridge', 'RandomForest'])
    
    if regressor_name == 'LinearRegression':
        regressor = LinearRegression()
    elif regressor_name == 'Ridge':
        alpha = trial.suggest_float('alpha', 1e-5, 10.0)
        regressor = Ridge(alpha=alpha)
    elif regressor_name == 'RandomForest':
        max_depth = trial.suggest_int('max_depth', 2, 32)
        n_estimators = trial.suggest_int('n_estimators', 10, 300)
        regressor = RandomForestRegressor(max_depth=max_depth, n_estimators=n_estimators)
    
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', regressor)])
    
    scores = cross_val_score(pipeline, X_train_reg, y_train_reg, cv=3, scoring='neg_mean_squared_error')
    mse = -scores.mean()
    
    return mse

In [ ]:
study_regression = optuna.create_study(direction='minimize')
study_regression.optimize(objective_regression, n_trials=50)

[I 2024-09-03 19:09:46,327] A new study created in memory with name: no-name-1ad1cb9f-5aaa-495b-aca5-874f6a8c1d1d
[I 2024-09-03 19:09:50,863] Trial 0 finished with value: 270.361038182287 and parameters: {'regressor': 'Ridge', 'alpha': 4.217166112598749}. Best is trial 0 with value: 270.361038182287.
[I 2024-09-03 19:09:55,439] Trial 1 finished with value: 270.360789788639 and parameters: {'regressor': 'Ridge', 'alpha': 7.209622806540318}. Best is trial 1 with value: 270.360789788639.
[I 2024-09-03 19:18:43,393] Trial 2 finished with value: 236.58478583543499 and parameters: {'regressor': 'RandomForest', 'max_depth': 27, 'n_estimators': 87}. Best is trial 2 with value: 236.58478583543499.
[I 2024-09-03 19:18:48,760] Trial 3 finished with value: 270.3627746236077 and parameters: {'regressor': 'LinearRegression'}. Best is trial 2 with value: 236.58478583543499.
[I 2024-09-03 19:27:00,560] Trial 4 finished with value: 236.31792702076356 and parameters: {'regressor': 'RandomForest', 'max_d

In [ ]:
best_params_reg = study_regression.best_trial.params
if best_params_reg['regressor'] == 'LinearRegression':
    best_model_reg = LinearRegression()
elif best_params_reg['regressor'] == 'Ridge':
    best_model_reg = Ridge(alpha=best_params_reg['alpha'])
else:
    best_model_reg = RandomForestRegressor(max_depth=best_params_reg['max_depth'], n_estimators=best_params_reg['n_estimators'])

In [ ]:
pipeline_regression = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', best_model_reg)])
pipeline_regression.fit(X_train_reg, y_train_reg)

In [ ]:
y_pred_reg = pipeline_regression.predict(X_test_reg)
mse_reg = mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print(f'Mean Squared Error (Регресія): {mse_reg}')
print(f'R^2 Score (Регресія): {r2_reg}')

In [ ]:
joblib.dump(pipeline_regression, 'best_regression_model.pkl')

X = df_cleaned.drop(columns=['Stay_Days', 'Stay'])
y_classification = df_cleaned['Stay']

X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X, y_classification, test_size=0.2, random_state=42)

In [ ]:
def objective_classification(trial):
    classifier_name = trial.suggest_categorical('classifier', ['LogisticRegression', 'RandomForest'])
    
    if classifier_name == 'LogisticRegression':
        C = trial.suggest_float('C', 1e-5, 10.0)
        classifier = LogisticRegression(C=C, max_iter=1000)
    elif classifier_name == 'RandomForest':
        max_depth = trial.suggest_int('max_depth', 2, 32)
        n_estimators = trial.suggest_int('n_estimators', 10, 300)
        classifier = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators)
    
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', classifier)])
    
    scores = cross_val_score(pipeline, X_train_cls, y_train_cls, cv=3, scoring='accuracy')
    accuracy = scores.mean()
    
    return accuracy

study_classification = optuna.create_study(direction='maximize')
study_classification.optimize(objective_classification, n_trials=50)

best_params_cls = study_classification.best_trial.params
if best_params_cls['classifier'] == 'LogisticRegression':
    best_model_cls = LogisticRegression(C=best_params_cls['C'], max_iter=1000)
else:
    best_model_cls = RandomForestClassifier(max_depth=best_params_cls['max_depth'], n_estimators=best_params_cls['n_estimators'])

pipeline_classification = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', best_model_cls)])
pipeline_classification.fit(X_train_cls, y_train_cls)

In [ ]:
y_pred_cls = pipeline_classification.predict(X_test_cls)
accuracy_cls = accuracy_score(y_test_cls, y_pred_cls)
f1_cls = f1_score(y_test_cls, y_pred_cls, average='weighted')

print(f'Accuracy (Класифікація): {accuracy_cls}')
print(f'F1 Score (Класифікація): {f1_cls}')

In [ ]:
joblib.dump(pipeline_classification, 'best_classification_model.pkl')